In [0]:
from pyspark.sql.functions import sum as _sum, avg as _avg

sap_silver = spark.table("enterprise_modernization.silver.silver_sap")
crm_silver = spark.table("enterprise_modernization.silver.silver_crm")
fleet_silver = spark.table("enterprise_modernization.silver.silver_fleet")

# Aggregate fleet data
fleet_agg = fleet_silver.groupBy("vehicle_id").agg(
    _sum("fault_detected").alias("fault_count"),
    _avg("odometer").alias("avg_odometer")
)

# Join with CRM and SAP
gold_df = crm_silver.join(
    sap_silver, "vehicle_id", "left"
).join(
    fleet_agg, "vehicle_id", "left"
)

gold_df.write.mode("overwrite").saveAsTable("enterprise_modernization.gold.customer_vehicle_fleet")
